In [2]:

!pip install triton --upgrade

In [10]:
# 1. Instalar o actualizar PyTorch con soporte CUDA (ejemplo para CUDA 12.1)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# 2. Instalar y actualizar los paquetes de Hugging Face
!pip install transformers accelerate huggingface-hub --upgrade

# 3. Instalar bitsandbytes (para la cuantificación de 8 bits)
!pip install bitsandbytes

Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached huggingface_hub-1.0.0rc6-py3-none-any.whl.metadata (14 kB)
Using cached huggingface_hub-1.0.0rc6-py3-none-any.whl (502 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0


In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import pipeline
import torch
import numpy as np

/home/marcelino.maita/.conda/envs/hf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import triton
print(triton.__version__)

3.5.0


In [4]:
pip install bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 16.3 MB/s  0:00:03m0:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
RANDOM_STATE = 42
path_DATA = "../../../data"
csv_path = f"{path_DATA}/spotify_dataset_sin_duplicados_4.csv"

TESTING = True

if TESTING:
    NROWS = 50
else:
    NROWS = None
    


In [8]:
def get_array(path):
    with open(path, "r") as f:
        array_ = json.load(f)  
    return array_

def get_song_and_target(csv_path, faltantes_path = "faltantes_according_token.json", sample_size=None):
    df = pd.read_csv(csv_path, nrows=sample_size)
    indices_to_remove = get_array(faltantes_path)
    if indices_to_remove[-1]>df.shape[0]:
        print("You are executing in testing way")
    else:
        df = df.drop(indices_to_remove).reset_index(drop=True)
    X = df['text']
    df['Explicit_binary'] = (df['Explicit'].str.lower() == 'yes').astype(int)
    y = df['Explicit_binary']
    return X, y

In [9]:
X,y = get_song_and_target(csv_path,"faltantes_according_token.json", sample_size=NROWS)

print(X.shape)
print(y.shape)

You are executing in testing way
(50,)
(50,)


gpt_oss_20b

In [3]:

pip install triton

Note: you may need to restart the kernel to use updated packages.


In [1]:
from transformers import pipeline
import torch

model_id = "openai/gpt-oss-20b"

pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.float16,
    device_map="auto",
)

messages = [
    {"role": "user", "content": "Explain quantum mechanics clearly and concisely."},
]

outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])


/home/marcelino.maita/.conda/envs/hf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Unrecognized keys in `rope_parameters` for 'rope_type'='yarn': {'truncate'}
Unrecognized keys in `rope_parameters` for 'rope_type'='yarn': {'truncate'}
`torch_dtype` is deprecated! Use `dtype` instead!
Unrecognized keys in `rope_parameters` for 'rope_type'='yarn': {'truncate'}
Unrecognized keys in `rope_parameters` for 'rope_type'='yarn': {'truncate'}
MXFP4 quantization requires Triton and kernels installed: CUDA requires Triton >= 3.4.0, XPU requires Triton >= 3.5.0, we will default to dequantizing the model to bf16
Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.24s/it]


RuntimeError: expected scalar type Half but found BFloat16

In [1]:
from transformers import pipeline, BitsAndBytesConfig
import torch
import bitsandbytes
model_id = "openai/gpt-oss-20b"

# --- Configuración para forzar la carga en 8 bits (INT8) ---
# Esto ofrece un excelente equilibrio entre ahorro de VRAM y calidad del modelo.
int8_config = BitsAndBytesConfig(
    load_in_8bit=True,
    # El resto de parámetros se pueden omitir, ya que la librería
    # usará automáticamente los kernels de 8 bits.
)
# -----------------------------------------------------------

pipe = pipeline(
    "text-generation",
    model=model_id,
    quantization_config=int8_config, 
    # Usar torch.float16 aquí es común para los cálculos intermedios, 
    # pero a menudo es inferido por la config de 8 bits si no se especifica.
    # Podemos añadirlo explícitamente para mayor control.
    torch_dtype=torch.float16, 
    device_map="auto",
)

# ... El resto de tu código de inferencia ...
messages = [
    {"role": "user", "content": "Explain quantum mechanics clearly and concisely."},
]

outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"])

/home/marcelino.maita/.conda/envs/hf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Unrecognized keys in `rope_parameters` for 'rope_type'='yarn': {'truncate'}
Unrecognized keys in `rope_parameters` for 'rope_type'='yarn': {'truncate'}
`torch_dtype` is deprecated! Use `dtype` instead!
Unrecognized keys in `rope_parameters` for 'rope_type'='yarn': {'truncate'}
Unrecognized keys in `rope_parameters` for 'rope_type'='yarn': {'truncate'}
MXFP4 quantization requires Triton and kernels installed: CUDA requires Triton >= 3.4.0, XPU requires Triton >= 3.5.0, we will default to dequantizing the model to bf16
Loading checkpoint shards: 100%|██████████| 3/3 [00:44<00:00, 14.74s/it]


ValueError: The following `model_kwargs` are not used by the model: ['quantization_config'] (note: typos in the generate arguments will also show up in this list)

In [13]:
# X_train, X_test, y_train, y_test = train_test_split(
#     X,
#     y,
#     test_size=0.2,
#     stratify=y,
#     random_state=RANDOM_STATE
# )

# print("X_train.shape:", X_train.shape)
# print("y_test.shape:", y_train.shape)


hypothesis_template = "This lyric contains {} content"
classes_verbalized = ["explicit", "not explicit"]

device = 0 if torch.cuda.is_available() else -1

zeroshot_classifier = pipeline(
    "zero-shot-classification",
    model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0",
    device=device
)

train_scores = []
for lyric in X_train:
    output = zeroshot_classifier(
        lyric,
        classes_verbalized,
        hypothesis_template=hypothesis_template,
        multi_label=False
    )
    score_explicit = output['scores'][output['labels'].index('explicit')]
    # print(output['labels'])
    # print(output['scores'])
    # print(score_explicit)
    train_scores.append(score_explicit)
    

train_scores = np.array(train_scores)

NameError: name 'X_train' is not defined

In [6]:
thresholds = np.arange(0.1, 0.9, 0.05)
best_threshold = 0.5
best_f1 = 0

for t in thresholds:
    preds_binary = (train_scores >= t).astype(int)
    f1 = f1_score(y_train, preds_binary)
    # print(f"Threshold {t:.2f} -> F1: {f1:.4f}")
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = t

print("Mejor threshold:", best_threshold, "con F1:", best_f1)

Mejor threshold: 0.8000000000000002 con F1: 0.6363636363636364


In [8]:
test_scores = []
for lyric in X_test:
    output = zeroshot_classifier(
        lyric,
        classes_verbalized,
        hypothesis_template=hypothesis_template,
        multi_label=False
    )
    score_explicit = output['scores'][output['labels'].index('explicit')]
    test_scores.append(score_explicit)

test_scores = np.array(test_scores)
test_preds = (test_scores >= best_threshold).astype(int)

# Métricas
f1_test = f1_score(y_test, test_preds)
accuracy_test = accuracy_score(y_test, test_preds)
precision_test = precision_score(y_test, test_preds)
recall_test = recall_score(y_test, test_preds)

print(f"Accuracy en test: {accuracy_test:.8f}")
print(f"Precision en test: {precision_test:.8f}")
print(f"Recall en test: {recall_test:.8f}")
print(f"F1 en test: {f1_test:.8f}")

Accuracy en test: 0.80000000
Precision en test: 0.50000000
Recall en test: 1.00000000
F1 en test: 0.66666667


## Prueba con treshold 0.5

In [9]:
best_threshold = 0.5
print("Probando con: ",best_threshold )
test_scores = []
for lyric in X_test:
    output = zeroshot_classifier(
        lyric,
        classes_verbalized,
        hypothesis_template=hypothesis_template,
        multi_label=False
    )
    score_explicit = output['scores'][output['labels'].index('explicit')]
    test_scores.append(score_explicit)

test_scores = np.array(test_scores)
test_preds = (test_scores >= best_threshold).astype(int)

# Métricas
f1_test = f1_score(y_test, test_preds)
accuracy_test = accuracy_score(y_test, test_preds)
precision_test = precision_score(y_test, test_preds)
recall_test = recall_score(y_test, test_preds)

print(f"Accuracy en test: {accuracy_test:.8f}")
print(f"Precision en test: {precision_test:.8f}")
print(f"Recall en test: {recall_test:.8f}")
print(f"F1 en test: {f1_test:.8f}")

Probando con:  0.5
Accuracy en test: 0.60000000
Precision en test: 0.33333333
Recall en test: 1.00000000
F1 en test: 0.50000000
